In [1]:
import prosperity3bt
import pandas as pd
import itertools
import numpy as np

from prosperity3bt.runner import run_backtest
from prosperity3bt.models import TradeMatchingMode
from prosperity3bt.file_reader import FileSystemReader
from pathlib import Path

from importlib import reload

In [2]:
def day_profit(result):
    last_timestamp = result.activity_logs[-1].timestamp

    profit = 0
    for row in reversed(result.activity_logs):
        if row.timestamp != last_timestamp:
            break

        profit += row.columns[-1]

    return profit

def backtest_days(trader, days=[-2, -1, 0], data_path='.', round_num=1):
    profits = []
    for day in days:
        backtest_results = prosperity3bt.runner.run_backtest(trader, file_reader=FileSystemReader(Path(data_path)), round_num=round_num, day_num=day, print_output=False, trade_matching_mode=TradeMatchingMode.all, no_names=True, show_progress_bar=False)
        profit = day_profit(backtest_results)
        profits.append(profit)
    return profits

def generate_param_combinations(param_grid):
    param_names = param_grid.keys()
    param_values = param_grid.values()
    combinations = list(itertools.product(*param_values))
    return [dict(zip(param_names, combination)) for combination in combinations]

In [11]:
import ink_trader
reload(ink_trader)

param_grid = {
    'ink_change_threshold_pct': [0.015],
    'ink_window_size': [3, 5, 10, 12, 15, 20, 25],
    # 'ink_window_size': [10],
    'ink_position_limit': [50],
    'clear_price_thresh': [0]
    # 'ink_change_threshold_pct': [0.01, 0.012, 0.015, 0.02, 0.022],
    # 'ink_position_limit': [50],
    # 'ink_position_limit': [30, 35, 40, 45, 50],
    # 'clear_price_thresh': [0, 1, 2, 3, 5]
}

PARAMS = {
    'ink_change_threshold_pct': 0.02,
    'ink_window_size': 10,
    'ink_position_limit': 50,
}

param_profits = []

for params in generate_param_combinations(param_grid):
    trader = ink_trader.Trader(params)
    days = [-2, -1, 0]
    profits = backtest_days(trader, days=days, data_path='.', round_num=1)
    for d in days:
        params[f'profit_day_{d}'] = profits[days.index(d)]
    param_profits.append(params)

In [13]:
df = pd.DataFrame(param_profits)
df['tot_profit'] = df[[f'profit_day_{d}' for d in days]].sum(axis=1)
df.sort_values(by='tot_profit', ascending=False, inplace=True)
df

,ink_change_threshold_pct,ink_window_size,ink_position_limit,clear_price_thresh,profit_day_-2,profit_day_-1,profit_day_0,tot_profit
6,0.015,25,50,0,1127.0,8548.0,706.0,10381.0
0,0.015,3,50,0,4273.0,3517.0,0.0,7790.0
1,0.015,5,50,0,4273.0,3517.0,0.0,7790.0
2,0.015,10,50,0,4273.0,3517.0,0.0,7790.0
3,0.015,12,50,0,4273.0,3517.0,0.0,7790.0
5,0.015,20,50,0,1127.0,3576.0,418.5,5121.5
4,0.015,15,50,0,1127.0,3517.0,0.0,4644.0


In [17]:
df.iloc[0].drop([f'profit_day_{day}' for day in days] + ['tot_profit']).to_dict()

{'ink_change_threshold_pct': 0.015,
 'ink_window_size': 25.0,
 'ink_position_limit': 50.0,
 'clear_price_thresh': 0.0}

In [ ]:
df.to_csv('backtests/inkbt2.csv')